In [1]:
# Libraries for phills_part
import math
import random
from typing import Tuple

from numpy import pi
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import LogicalExpressionOracle
from qiskit.circuit import classical_function, Int1
from qiskit.circuit.library import IntegerComparator, GroverOperator
from qiskit.result import Counts

from utils import *

from qiskit import *
from qiskit.extensions import Initialize
from qiskit.visualization import plot_histogram

# Libraries for sams_part
import math
import pprint
import random
# importing Qiskit
from qiskit import *
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.extensions import Initialize

# Libraries from test_comparator.py
from unittest import TestCase
from comparator import *

import numpy as np
import copy


In [2]:
# Utils
def scalar(multiplicand, matrix):
    ret = copy.deepcopy(matrix)
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            ret[i][j] = multiplicand * matrix[i][j]
    return ret


# N number to convert to superposition
# returns super position in form of [[int]]
# i.e. N = 1 returns [[0],[1]]
# i.e. N = 2 returns [[0],[0],[1]]
def get_superposition(n, size):
    if size < 2:
        size = 2
    ret = []
    binary_string = format(n + 1, 'b').zfill(size)
    for i in range(len(binary_string) - 1, -1, -1):
        ret.append([int(binary_string[i])])
    return ret

In [3]:
# Phills_part
L = []
y = 0


def get_pi_perm(n):
    pi = {}
    set = []
    for i in range(n):
        set.append(i)

    set_len = int(len(set) / 2)
    for i in range(set_len):
        key = random.choice(set)
        set.remove(key)
        value = random.choice(set)
        set.remove(value)
        pi[key] = value
        pi[value] = key
    return pi


# N integer
def get_L(N):
    ret = []
    pi_perm = get_pi_perm(N)
    for i in range(len(pi_perm.keys())):
        ret.append(pi_perm.get(i))
    return ret


def run():
    global L, y
    N = 16
    L = get_L(N)
    y = random.choice(range(N))
    m = 10
    for i in range(m):
        number_of_qibits = math.ceil(math.log(N, 2))
        register = QuantumRegister(number_of_qibits)
        # qc = QuantumCircuit(register)
        # for i in range(number_of_qibits):
        #     qc.h(register[i])
        # # oracle.append(IntegerComparator(number_of_qibits, L[y]))
        # qc.z([0, 1, 2, 3])
        # qc.cz(0, 3)
        # qc.h([0, 1, 2, 3])

        oracle = QuantumCircuit(number_of_qibits)
        for i in range(number_of_qibits):
            oracle.h(i)

        # TODO append actual oracle part of the oracle here
        oracle.x(0)
        oracle.x(1)

        oracle.cp(pi / 4, 0, 3)
        oracle.cx(0, 1)
        oracle.cp(-pi / 4, 1, 3)
        oracle.cx(0, 1)
        oracle.cp(pi / 4, 1, 3)
        oracle.cx(1, 2)
        oracle.cp(-pi / 4, 2, 3)
        oracle.cx(0, 2)
        oracle.cp(pi / 4, 2, 3)

        oracle.x(0)
        oracle.x(1)
        oracle.cz(0, 1)

        grover_operator = GroverOperator(oracle=oracle)
        grover_operator.measure_all()
        backend = BasicAer.get_backend('qasm_simulator')
        results = execute(grover_operator, backend, shots=1000).result()
        counts = results.get_counts(grover_operator)
        print(counts)
        plot_histogram(counts)
        amplified_state = get_amplified_state(counts)
        y_primed = int("0b" + amplified_state[0], 2)
        print(f"amplified_state {amplified_state}")
        set_new_y(y_primed)


def set_new_y(y_primed):
    global L, y
    if L[y_primed] < L[y]:
        y = y_primed


def get_amplified_state(counts: Counts):
    highest = ('0', 0)
    for count in counts.items():
        if highest[1] < count[1]:
            highest = count
    return highest


def get_good_states(str):
    return ['1111']


def draw(qc: QuantumCircuit):
    qc.draw(output='mpl')


if __name__ == '__main__':
    run()


{'1011': 180, '0011': 178, '0111': 164, '1111': 188, '1100': 138, '0101': 11, '1010': 50, '1001': 12, '0000': 8, '0001': 10, '0110': 9, '0100': 6, '1110': 13, '1000': 18, '1101': 9, '0010': 6}
amplified_state ('1111', 188)
{'0011': 176, '1111': 194, '1011': 169, '1100': 143, '0110': 16, '0111': 171, '1010': 49, '1001': 6, '0001': 13, '1110': 19, '1101': 4, '0000': 7, '0100': 9, '0101': 8, '1000': 12, '0010': 4}
amplified_state ('1111', 194)
{'0100': 11, '1100': 143, '0111': 161, '0011': 181, '1000': 15, '1101': 12, '1011': 183, '1111': 179, '1110': 15, '1010': 50, '0110': 9, '1001': 10, '0001': 7, '0000': 11, '0101': 10, '0010': 3}
amplified_state ('1011', 183)
{'1011': 176, '1000': 21, '1111': 188, '0011': 177, '1100': 148, '0111': 155, '1110': 13, '0110': 11, '0100': 10, '0000': 8, '1010': 56, '0010': 9, '0101': 5, '0001': 7, '1101': 11, '1001': 5}
amplified_state ('1111', 188)
{'1100': 156, '1011': 179, '1111': 169, '1110': 20, '0011': 191, '0111': 164, '0101': 9, '1000': 16, '0000'

In [7]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

def oracle(lst, y):
    qc = QuantumCircuit(4, 1)
    #qc.reset(range(4))
    n1 = '0'
    n2 = '0'
    n3 = '0'
    n4 = '0'


    if qc.measure(0,0)=='1':
        n1 = '1'
        qc.x(3)
    if qc.measure(1,0)=='1':
        n2 = '1'
        qc.x(2)
    if qc.measure(2,0)=='1':
        n3 = '1'
        qc.x(1)
    if qc.measure(3,0)=='1':
        n4 = '1'
        qc.x(0)

    # barrier between input state and gate operation
    qc.barrier()

    #n_bin_string = "0b" + n1 + n2 + n3 + n4
    #print('n bin string: ', n_bin_string)

    if (lst[int(n_bin_string,2)] < lst[y]):
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
        qc.measure_all()

    else:
        qc.measure_all()

    qc.barrier()

    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1, memory=True)
    output = job.result().get_memory()[0]

    return qc, output

def num_comp(n1,n2,n3,n4,y1,y2,y3,y4,lst):

    qc = QuantumCircuit(4, 1)
    qc.reset(range(4))

    if n1==1:
        qc.x(3)
    if n2==1:
        qc.x(2)
    if n3==1:
        qc.x(1)
    if n4==1:
        qc.x(0)

    # barrier between input state and gate operation
    qc.barrier()

    n_bin_string = "0b" + str(n1) + str(n2) + str(n3) + str(n4)
    print('n bin string: ', n_bin_string)
    y_bin_string = "0b" + str(y1) + str(y2) + str(y3) + str(y4)
    print('y bin string: ', y_bin_string)

    if (lst[int(n_bin_string,2)] < lst[int(y_bin_string,2)]):
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
        qc.measure_all()
    else:
        qc.measure_all()

    qc.barrier()

    #We'll run the program on a simulator
    backend = Aer.get_backend('qasm_simulator')
    #Since the output will be deterministic, we can use just a single shot to get it
    job = execute(qc, backend, shots=1, memory=True)
    output = job.result().get_memory()[0]

    return qc, output

def grover():
    n = 4
    grover_circuit = QuantumCircuit(n)
    number_of_qibits = math.ceil(math.log(n))
    L = get_L(16)
    y = random.choice(range(n))

    print(L)

    grover_circuit = initialize_s(grover_circuit, [0,1,2,3])

    #comparator = IntegerComparator(num_state_qubits=number_of_qibits, value=1, geq=True, name='cmp')

    qc_comp, out = num_comp(0,0,0,1,0,0,1,0,L)
    print(qc_comp)
    grover_circuit.append(qc_comp, range(n))

    grover_circuit.h([0,1,2,3])
    grover_circuit.z([0,1,2,3])
    grover_circuit.cz(0,3)
    grover_circuit.h([0,1,2,3])

    print('Comparing ',L[1],' < ',L[2],' gives output',out)

    print(grover_circuit)
    
grover()

[5, 4, 3, 2, 1, 0, 11, 8, 7, 12, 14, 6, 9, 15, 10, 13]
n bin string:  0b0001
y bin string:  0b0010
             ┌───┐ ░  ░ ┌─┐          ░ 
   q_0: ─|0>─┤ X ├─░──░─┤M├──────────░─
             └───┘ ░  ░ └╥┘┌─┐       ░ 
   q_1: ─|0>───────░──░──╫─┤M├───────░─
                   ░  ░  ║ └╥┘┌─┐    ░ 
   q_2: ─|0>───────░──░──╫──╫─┤M├────░─
                   ░  ░  ║  ║ └╥┘┌─┐ ░ 
   q_3: ─|0>───────░──░──╫──╫──╫─┤M├─░─
                   ░  ░  ║  ║  ║ └╥┘ ░ 
   c: 1/═════════════════╬══╬══╬══╬════
                         ║  ║  ║  ║    
meas: 4/═════════════════╩══╩══╩══╩════
                         0  1  2  3    
Comparing  4  <  3  gives output 0001 0
     ┌───┐┌──────────────┐┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤0             ├┤ H ├┤ Z ├─■─┤ H ├
     ├───┤│              │├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤1             ├┤ H ├┤ Z ├─┼─┤ H ├
     ├───┤│  circuit-562 │├───┤├───┤ │ ├───┤
q_2: ┤ H ├┤2             ├┤ H ├┤ Z ├─┼─┤ H ├
     ├───┤│              │├───┤├───┤ │ ├───┤
q_3: ┤ H ├┤3             ├